# Зависимости



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Веселье с PyLDAVis

In [2]:
! pip install numpy
! pip install pymorphy2[fast]
! pip install wikipedia
! pip install nltk
! pip install rake-nltk
! pip install transformers
! pip install -update pandas
! pip install pyLDAvis


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [25]:
!pip install pymorphy2[fast]

In [26]:
import wikipedia
import random
import nltk
import re
import string
import threading
import multiprocessing
import os
import shutil
from nltk.corpus import stopwords
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import ToTensor
import warnings 
from sklearn.linear_model import LogisticRegression
from PIL import Image
from IPython.display import clear_output
from typing import List
import nltk
import string
import pymorphy2
import codecs

In [28]:
warnings.filterwarnings(action='ignore',category=UserWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='gensim')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='smart_open') 
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='sklearn')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='scipy')    
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='pymorphy2')

In [5]:
nltk.download('stopwords')
nltk.download('punkt')
stopWordsRu = set(stopwords.words("russian")) 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
all_data = pd.read_csv('/content/drive/MyDrive/aiijc_sber/tokens_full_cleaned.csv')
all_data.head()

,Unnamed: 0,texts,y,tokens
0,0,Живо́тные (лат. Animalia) — традиционно (со вр...,0,живо тный лата аристотель выделять организм би...
1,1,Пти́цы (лат. Aves) — группа теплокровных яйцек...,0,пти цы лата группа теплокровный яйцекладущий п...
2,2,Союз охраны птиц России (СОПР) — общероссийска...,0,союз охрана птица россия сопра общероссийский ...
3,3,Крючковидные отростки рёбер — костяные выросты...,0,крючковидный отросток ребро костяной вырост ве...
4,4,Myxozoa (лат.) — класс или подтип паразитическ...,0,лата класс подтип паразитический беспозвоночны...


In [15]:
dop = pd.read_csv('/content/drive/MyDrive/aiijc_sber/full_marked_dataset.csv')
dop = dop[dop.ans!='неизвестно']
dop.head()

,Unnamed: 0,text,ans
0,0,\t1.1. Летом в спортивный лагерь ходили 50 дет...,спорт
1,1,\t1.2. На концерте в летнем лагере ребята игра...,музыка
3,3,\t1.4. Во время летних соревнований по плавани...,спорт
4,4,\t1.5. В спортивную школу во время летних кани...,спорт
6,6,\t1.7. Самые крупные животные на нашей планете...,животные


In [16]:
facts2 = pd.read_csv('/content/drive/MyDrive/aiijc_sber/facts2.csv')
mit_dict = {0:"животные",
            1: "музыка",
            2: "спорт",
            3: "литература"}
facts2.labels = [mit_dict[i] for i in facts2.labels]
facts2.head()

,texts,labels
0,1 Хамелеоны могут двигать глазами в разных нап...,животные
1,2 Белка – лучший садовник. Миллионы деревьев в...,животные
2,3 Зуб слона может весить до девяти килограмм!,животные
3,"4 У млекопитающих кровь красная, у насекомых ж...",животные
4,5 В среднем коровы какают 16 раз в день.,животные


In [17]:
data_texts = all_data.texts.to_list() + dop.text.to_list() + facts2.texts.to_list()
data_ans = [['животные', "музыка", "спорт", "литература"][i] for i in all_data.y] + dop.ans.to_list() + facts2.labels.to_list()
data = pd.DataFrame()
data['text'] = data_texts
data['ans'] = data_ans
data.head()

,text,ans
0,Живо́тные (лат. Animalia) — традиционно (со вр...,животные
1,Пти́цы (лат. Aves) — группа теплокровных яйцек...,животные
2,Союз охраны птиц России (СОПР) — общероссийска...,животные
3,Крючковидные отростки рёбер — костяные выросты...,животные
4,Myxozoa (лат.) — класс или подтип паразитическ...,животные


In [22]:
class Preparator():
  def __init__(self):
      self.morph = pymorphy2.MorphAnalyzer()
      self.tokenizer = nltk.WordPunctTokenizer()
      self.stopwords = set(line.strip() for line in codecs.open('/content/drive/MyDrive/aiijc_sber/all_stopwords.txt', "r", "utf_8_sig").readlines())
      self.bigram_mod = gensim.models.Phrases.load("/content/drive/MyDrive/aiijc_sber/bigram_model.pkl")

  def prepare_corp(self, news_list: List[str]):
      return [self.newstext2token(news_text) for news_text in news_list]

  def make_bigrams(self, doc):
      return self.bigram_mod[doc]

  def newstext2token(self, news_text: str):
      tokens = self.tokenizer.tokenize(news_text.lower())
      tokens_with_no_punct = [self.morph.parse(w)[0].normal_form for w in tokens if all(c in 'йцукенгшщзхъёфывапролджэячсмитьбю' for c in w)]
      tokens_base_forms = [w for w in tokens_with_no_punct if w not in self.stopwords]
      tokens_last = [w for w in tokens_base_forms if len(w)>1]
      tokens_bigrammed = self.make_bigrams(tokens_last)
      return tokens_bigrammed

  

In [128]:
prep = Preparator()
all_texts = data.text
print(len(all_texts))
all_tokens = []
for i in range(len(all_texts)):
  all_tokens.append(prep.newstext2token(all_texts[i]))
  if i%1000 == 0:
    print(i)
data['tokens'] = all_tokens

3658
0
1000
2000
3000


In [121]:
category2id = ['животные', 'музыка', 'литература', 'спорт']
total_matrix = {}
for doc in range(len(data.tokens)):
  theme = data.ans[doc]
  for word in data.tokens[doc]:
    if word not in total_matrix:
      total_matrix[word] = [0, 0, 0, 0]
    total_matrix[word][category2id.index(theme)] += 1

In [122]:
themes_representation = {'животные': [], 'музыка': [], 'литература': [], 'спорт': []}
for word in total_matrix:
  word_theme = np.array(total_matrix[word]).argmax()
  if total_matrix[word][word_theme]/sum(total_matrix[word]) > 0.5:
    themes_representation[category2id[word_theme]].append((word, (total_matrix[word][word_theme]**2)/sum(total_matrix[word])))

In [123]:
for theme in themes_representation:
  themes_representation[theme] = sorted(themes_representation[theme], key = lambda x: -x[1])

In [125]:
themes_representation['музыка']

[('музыка', 2437.5187517109225),
 ('музыкальный', 1625.0054559625876),
 ('композитор', 754.3129200896191),
 ('инструмент', 742.1449912126537),
 ('моцарт', 730.296052631579),
 ('звук', 657.2800361336947),
 ('кошка', 637.1115335868187),
 ('собака', 624.6448497854077),
 ('нота', 575.7202380952381),
 ('прокофьев', 504.36842105263156),
 ('тональность', 480.2),
 ('логика', 346.9879518072289),
 ('октав', 304.5121212121212),
 ('символ', 302.64234449760767),
 ('фортепиано', 302.2012578616352),
 ('гармония', 276.8022988505747),
 ('нотация', 276.0874125874126),
 ('интервал', 268.2470588235294),
 ('музыкант', 262.8888888888889),
 ('частота', 258.93690248565963),
 ('мелодия', 247.54371584699453),
 ('концерт', 242.79775280898878),
 ('поисковый', 231.33444816053512),
 ('симфония', 231.21804511278197),
 ('глагол', 224.93076923076924),
 ('бетховен', 220.3403361344538),
 ('буква', 210.37826086956522),
 ('оркестр', 206.4516129032258),
 ('ритм', 204.85344827586206),
 ('грамматика', 200.64285714285714),
 (

In [127]:
with open("/content/drive/MyDrive/aiijc_sber/keywords_music.txt", 'w') as dump:
  for word in themes_representation['музыка']:
    dump.write(word[0]+'\n')
with open("/content/drive/MyDrive/aiijc_sber/keywords_literature.txt", 'w') as dump:
  for word in themes_representation['литература']:
    dump.write(word[0]+'\n')
with open("/content/drive/MyDrive/aiijc_sber/keywords_animals.txt", 'w') as dump:
  for word in themes_representation['животные']:
    dump.write(word[0]+'\n')
with open("/content/drive/MyDrive/aiijc_sber/keywords_sport.txt", 'w') as dump:
  for word in themes_representation['спорт']:
    dump.write(word[0]+'\n')



# Nouns only

In [129]:
morph = pymorphy2.MorphAnalyzer()

In [134]:
def write_nouns(filename, category):
  with open("/content/drive/MyDrive/aiijc_sber/keywords_nouns_" + filename + ".txt", 'w') as dump:
    for word in themes_representation[category]:
      if morph.parse(word[0])[0].tag.POS == "NOUN":
        dump.write(word[0]+'\n')


In [135]:
write_nouns('music', 'музыка')
write_nouns('literature', 'литература')
write_nouns('sport', 'спорт')
write_nouns('animals', 'животные')

# Actors


In [144]:
def write_nouns_actors(filename, category):
  with open("/content/drive/MyDrive/aiijc_sber/keywords_nouns_actors_" + filename + ".txt", 'w') as dump:
    for word in themes_representation[category]:
      if morph.parse(word[0])[0].tag.POS == "NOUN" and morph.parse(word[0])[0].tag.animacy == morph.parse('музыкант')[0].tag.animacy:
        dump.write(word[0]+'\n')


In [145]:
write_nouns_actors('music', 'музыка')
write_nouns_actors('literature', 'литература')
write_nouns_actors('sport', 'спорт')
write_nouns_actors('animals', 'животные')